In [1]:
from torchtext import data
import torch
import torch.nn as nn
import torch.optim as optim
import random
import datetime
from math import floor

seed = 1
torch.manual_seed(seed)
device = torch.device('cuda')
iterator_device = 0
##uncomment for cpu
#device = torch.device('cpu')
#iterator_device = -1

In [2]:
blurb_field = data.Field(sequential=True, use_vocab=True, lower=True, tokenize="spacy", include_lengths=True)
state_field = data.LabelField(sequential=False, use_vocab=False, tensor_type=torch.FloatTensor,
                              preprocessing=lambda x:1 if x=='successful' else 0)
dataset = data.TabularDataset(path='df_text_eng.csv',format='csv',skip_header=True,fields=[('Unnamed: 0', None),('blurb', blurb_field),('state', state_field)])

In [3]:
train, test, validation = dataset.split(random_state=random.seed(seed), split_ratio=[70,15,15])
print("Training Set Size: ", len(train))
print("Test Set Size: ", len(test))
print("Validation Set Size: ", len(validation))

Training Set Size:  150859
Test Set Size:  32327
Validation Set Size:  32327


In [7]:
##randomly
#words that appear less than 3 times (2 or less) will be considered unknown words with tag "<unk>", 
#they will have the same word embedding
#vocabulary of the training set will only be used to emulate real world situtaions when the test set is unknown
blurb_field.build_vocab(train,min_freq=3,vectors="glove.6B.100d")
print("Vocabulary size used: ",len(blurb_field.vocab))

Vocabulary size used:  28760


In [16]:
batch_size = 64

train_iter = data.BucketIterator(dataset=train, batch_size=batch_size, sort_key=lambda x:len(x.blurb),device=iterator_device,
                                repeat=False, train=True, sort_within_batch=True)
test_iter = data.BucketIterator(dataset=test, batch_size=batch_size, sort_key=lambda x:len(x.blurb),device=iterator_device,
                                train=False, sort_within_batch=True)
validation_iter = data.BucketIterator(dataset=validation, batch_size=batch_size, sort_key=lambda x:len(x.blurb),device=iterator_device,
                                      train=False, sort_within_batch=True)

In [34]:
class GRUNet(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size, padding_idx=None, init_embedding=None):
        super(GRUNet, self).__init__()
        if (init_embedding is not None):
            self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=padding_idx, _weight=init_embedding)    # word embedding
        else:
            self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=padding_idx)    # word embedding
        self.gru = nn.GRU(embed_size, hidden_size)  #GRU layer, default activation function is tanh
        self.out = nn.Linear(hidden_size, output_size)   # output layer (Fully Connected)
        

    def forward(self, x, x_lengths):
        #x -> [sentence length(max, smaller sentences are padded), minibatch size]
        embeds = self.embedding(x)
        
        #embeds -> [sentence length, minibatch size, embedding size]
        embeds = nn.utils.rnn.pack_padded_sequence(embeds,x_lengths)
        all_hiddens, last_hidden = self.gru(embeds)      # activation function for hidden layer
        #the first dimension of the hidden layer is 1 (we use one hidden layer uniderectional GRUs), squeeze removes it to be 
        #able to enter the fully connected layer with dimensions [batch size, hidden size]
        score = self.out(last_hidden.squeeze(0))           
        return score

In [35]:
def calc_accuracy(predictions, y):
    predictions = torch.round(torch.sigmoid(predictions))
    correct = (sum(predictions == y)).float()
    accuracy = correct/len(predictions)
    return accuracy

In [36]:
padding_idx = blurb_field.vocab.stoi['<pad>']
embed_size = 100
hidden_size = 100

In [37]:
grumodel = GRUNet(len(blurb_field.vocab),embed_size,hidden_size,1, padding_idx=padding_idx, init_embedding=blurb_field.vocab.vectors)
optimizer = optim.SGD(grumodel.parameters(), lr=1e-3)
loss_fn = nn.BCEWithLogitsLoss()
grumodel = grumodel.to(device)
loss_fn = loss_fn.to(device)

In [38]:
def train(model, iterator, optimizer, loss_fn):
    epoch_loss = 0
    epoch_accuracy = 0
    
    model.train()
    
    for minibatch in iterator:
        optimizer.zero_grad()
        predictions = model(minibatch.blurb[0],minibatch.blurb[1]).squeeze(1)
        loss = loss_fn(predictions, minibatch.state)
        accuracy = calc_accuracy(predictions, minibatch.state)    
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_accuracy += accuracy.item()
        
    return epoch_loss / len(iterator), epoch_accuracy / len(iterator)

In [39]:
def evaluate(model, iterator, loss_fn):
    epoch_loss = 0
    epoch_accuracy = 0
    
    model.eval()
    
    with torch.no_grad():
        for minibatch in iterator:
            predictions = model(minibatch.blurb[0],minibatch.blurb[1]).squeeze(1)
            loss = loss_fn(predictions, minibatch.state)
            accuracy = calc_accuracy(predictions, minibatch.state)    

            epoch_loss += loss.item()
            epoch_accuracy += accuracy.item()
        
    return epoch_loss / len(iterator), epoch_accuracy / len(iterator)

In [ ]:
no_epochs = 100
#import ipdb; ipdb.set_trace() # debugging starts here
train_accuracies = []
validation_accuracies = []
init_time = datetime.datetime.now()
for epoch in range(no_epochs):
    train_loss, train_accuracy = train(grumodel, train_iter, optimizer, loss_fn)
    validation_loss, validation_accuracy = evaluate(grumodel, validation_iter, loss_fn)
    train_accuracies.append(train_accuracy)
    validation_accuracies.append(validation_accuracy)
    current_time = datetime.datetime.now()
    total_time = (current_time-init_time).total_seconds()
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy*100:.2f}%, ' +  
          f'Validation Loss: {validation_loss:.4f}, Validation Accuracy: {validation_accuracy*100:.2f}%')
    print(f'Total Time Passed: {floor(total_time/3600)} hours, {floor(total_time/60)%60} minutes, {total_time%60:.2f} seconds')

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torchtext/data/field.py:321: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train), lengths
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torchtext/data/field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


Epoch: 01, Train Loss: 0.6932, Train Accuracy: 50.61%, Validation Loss: 0.6920, Validation Accuracy: 52.60%
Total Time Passed: 0 hours, 0 minutes, 22.08 seconds
Epoch: 02, Train Loss: 0.6915, Train Accuracy: 52.76%, Validation Loss: 0.6904, Validation Accuracy: 54.45%
Total Time Passed: 0 hours, 0 minutes, 44.10 seconds
Epoch: 03, Train Loss: 0.6901, Train Accuracy: 54.15%, Validation Loss: 0.6889, Validation Accuracy: 54.98%
Total Time Passed: 0 hours, 1 minutes, 6.10 seconds
Epoch: 04, Train Loss: 0.6889, Train Accuracy: 54.82%, Validation Loss: 0.6877, Validation Accuracy: 55.82%
Total Time Passed: 0 hours, 1 minutes, 28.00 seconds
Epoch: 05, Train Loss: 0.6878, Train Accuracy: 55.41%, Validation Loss: 0.6865, Validation Accuracy: 56.16%
Total Time Passed: 0 hours, 1 minutes, 49.93 seconds
Epoch: 06, Train Loss: 0.6868, Train Accuracy: 55.67%, Validation Loss: 0.6854, Validation Accuracy: 56.46%
Total Time Passed: 0 hours, 2 minutes, 11.77 seconds
Epoch: 07, Train Loss: 0.6858, Trai

In [ ]:
test_loss, test_accuracy = evaluate(grumodel, test_iter, loss_fn)

print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy*100:.2f}%')

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1,no_epochs+1),train_accuracies)
plt.show()

In [ ]:
with open('GRUModelGloveTraining.txt', 'w') as f:
    for item in train_accuracies:
        f.write("%s\n" % item)

In [ ]:
with open('GRUModelGloveVal.txt', 'w') as f:
    for item in validation_accuracies:
        f.write("%s\n" % item)